<a href="https://colab.research.google.com/github/rsanandres/Coursera/blob/master/Copy_of_Ensemble_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17146502003186916456
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13840458880352072570
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=c84d17b5bc073d75fbc7e586c3ce14253714eb29acfb7688dc1fc1d349e2a7a1
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


IndexError: ignored

In [2]:
import os 

In [3]:
os.chdir("drive/Shared drives")
os.chdir("affinity")

In [ ]:
!ls

In [4]:
#imports

import numpy as np
from time import time
import logging
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold

from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.metrics import f1_score, classification_report

from sklearn.pipeline import make_pipeline

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [5]:
from scripts.extract_data import extract_data

In [6]:
df = extract_data("data/news/")
df.head()

[('articles',), ('news_labels',)]
Complete!


,Source,Content,Label
0,Addicting Info,Since Donald Trump Jr. has admitted on Twitter...,left_bias
1,Addicting Info,The Republican-written and Republican-altered ...,left_bias
2,Addicting Info,The world is still laughing at the Trump admin...,left_bias
3,BBC,Visiting the Matanuska banana plantation is no...,left_center_bias
4,BBC,"In Croatia, thousands of people with mental il...",left_center_bias


In [10]:
df['Label'].unique()

array(['left_bias', 'left_center_bias', 'questionable_source',
       'right_bias', None, 'conspiracy_pseudoscience', 'least_biased',
       'right_center_bias', 'satire'], dtype=object)

In [7]:
bias_dictionary = {"left_bias": 0, "left_center_bias": 1, "least_biased": 2, "right_center_bias": 3, "right_bias": 4, 
                  "questionable_source": None, "conspiracy_pseudoscience": None, "satire": None, "Pro-Science": None}

df['Label'] = df['Label'].map(bias_dictionary)
cleaned_df = df
cleaned_df.head()

,Source,Content,Label
0,Addicting Info,Since Donald Trump Jr. has admitted on Twitter...,0.0
1,Addicting Info,The Republican-written and Republican-altered ...,0.0
2,Addicting Info,The world is still laughing at the Trump admin...,0.0
3,BBC,Visiting the Matanuska banana plantation is no...,1.0
4,BBC,"In Croatia, thousands of people with mental il...",1.0


In [29]:
cleaned_df.size

2140602

In [8]:
cleaned_df = cleaned_df.dropna().reset_index()

In [31]:
cleaned_df.size

1903112

In [9]:
cleaned_df['Label'].unique()

array([0., 1., 4., 2., 3.])

In [33]:
cleaned_df.head()

,index,Source,Content,Label
0,0,Addicting Info,Since Donald Trump Jr. has admitted on Twitter...,0.0
1,1,Addicting Info,The Republican-written and Republican-altered ...,0.0
2,2,Addicting Info,The world is still laughing at the Trump admin...,0.0
3,3,BBC,Visiting the Matanuska banana plantation is no...,1.0
4,4,BBC,"In Croatia, thousands of people with mental il...",1.0


In [9]:
scaler = MinMaxScaler()

In [10]:
cleaned_df = cleaned_df.drop(['index'], axis=1)
cleaned_df

,Source,Content,Label
0,Addicting Info,Since Donald Trump Jr. has admitted on Twitter...,0.0
1,Addicting Info,The Republican-written and Republican-altered ...,0.0
2,Addicting Info,The world is still laughing at the Trump admin...,0.0
3,BBC,Visiting the Matanuska banana plantation is no...,1.0
4,BBC,"In Croatia, thousands of people with mental il...",1.0
...,...,...,...
475773,iPolitics,'Scheer is now stuck: If he remains relatively...,1.0
475774,iPolitics,The Supreme Court of Canada has upheld an orde...,1.0
475775,iPolitics,"Alberta plans to buy about 7,000 railcars to h...",1.0
475776,iPolitics,Furniture giant IKEA is making changes to its ...,1.0


In [19]:
#idk why i did this i don't think i should scale it 

#scaled_ratings = scaler.fit_transform(cleaned_df[['Label']])
#scaled_ratings.size
#scaled_ratings = pd.DataFrame(scaled_ratings, columns=['Label'])
#scaled_ratings.head()

In [20]:
#cleaned_df = pd.concat([cleaned_df['Content'], scaled_ratings['Label']], axis=1)

In [11]:
cleaned_df = cleaned_df.sample(n = 200000)

In [12]:
tfidf_transformer = TfidfVectorizer(max_features = 2000, stop_words='english', ngram_range=(2,2))
tfidf = tfidf_transformer.fit_transform(cleaned_df['Content'])

#X = pd.concat([pd.DataFrame(tfidf.toarray(), columns=tfidf_transformer.get_feature_names()), cleaned_df['Label']], axis=1, ignore_index=True)
X = pd.DataFrame(tfidf.toarray(), columns=tfidf_transformer.get_feature_names())
y = cleaned_df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [28]:
X_train.shape
X_train


,000 000,000 people,000 votes,000 year,0207 782,02077824220 pay,07810 791,10 000,10 days,10 million,10 minutes,10 percent,10 year,10 years,100 000,100 cent,100 million,100 percent,100 years,11 year,12 000,12 months,12 year,12 years,130 000,14 year,14 years,15 000,15 minutes,15 percent,15 year,15 years,150 000,16 year,16 years,17 people,17 year,17 years,18 09,18 10,...,world champion,world cup,world leaders,world trade,world war,worlds largest,worth noting,wrote letter,wrote twitter,www telesurtv,www thesun,xi jinping,yahoo news,year according,year ago,year half,year later,year old,year olds,year said,year trump,year year,years ago,years come,years later,years old,years prison,years said,york city,york state,york times,young children,young man,young men,young people,young woman,young women,youre going,youve got,zero tolerance
195436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
133504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.166378,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
193602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.316143,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.371038,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4012,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
29919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
21642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.01431,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
190083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
77049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [29]:
y_train.shape
y_train

291451    0.0
252089    3.0
472835    1.0
18806     4.0
41402     1.0
         ... 
300859    4.0
264750    0.0
65958     0.0
349540    1.0
474252    3.0
Name: Label, Length: 150000, dtype: float64

In [55]:
pipeline = Pipeline([
    ('tfidf',  TfidfVectorizer()),
    ('clf', LogisticRegression()),
])

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (2,2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    #'clf__max_iter': (20,),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X, y)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'tfidf__use_idf': (True, False),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__max_features': (None, 5000, 10000, 50000),
 'vect__ngram_range': ((1, 1), (1, 2), (2, 2))}
Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: ignored

In [ ]:
pipe = make_pipeline(TfidfVectorizer(), LogisticRegression())
param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100], "tfidfvectorizer__ngram_range": [(1,1), (1,2), (1,3)]}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

In [ ]:
X

,000,10,100,11,12,13,14,15,16,17,18,19,20,200,2008,2009,2010,2011,2012,2013,2014,2015,2016,2016 election,2017,2018,2019,2020,21,22,23,24,25,26,27,28,29,30,300,31,...,winning,woman,women,won,wont,word,words,work,worked,workers,working,works,world,world cup,worlds,worried,worse,worst,worth,wouldnt,write,writing,written,wrong,wrote,www,www thesun,year,year old,years,years ago,yes,yesterday,york,york times,young,youre,youtube,zero,zu
0,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.089475,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.091491,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.086901,0.0,0.000000,0.000000,0.0,0.0,0.067616,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.092697,0.000000,0.068810,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.188793,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.148589,0.0,0.00000,0.0,0.0,0.144377,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.107266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.113996,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.09301,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.060630,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.074153,0.0,0.00000,0.0,0.000000,0.0,0.071586,0.000000,0.0,0.0,0.000000,0.0,0.047791,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199996,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.104019,0.0,0.0,0.000000,0.000000,0.0000

In [ ]:
y

149464    3.0
325367    4.0
184213    1.0
431096    3.0
11830     3.0
         ... 
457260    3.0
207913    4.0
441527    1.0
177852    1.0
357840    3.0
Name: Label, Length: 200000, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [ ]:
# Defining Model
regressor = LogisticRegression(max_iter=500)
# Training Model
regressor.fit(X_train, y_train)
# Making Predictions
y_pred4 = regressor.predict(X_test)
# Evaluating
print("Accuracy Score: ", accuracy_score(y_test, y_pred4))


In [ ]:
y_probs = regressor.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_probs, average="macro", multi_class="ovo")
print('ROC Score is: ', roc_auc)


In [ ]:
#SVM might be hard with big datasets because of the computational difficulties of creating a gazillion new planes 

# Defining Model
clf = SVC(probability=True)
# Training Model
clf.fit(X_train, y_train)
# Making Predictions
y_pred5 = clf.predict(X_test)
# Evaluating
print("Accuracy Score: ", accuracy_score(y_test, y_pred5))

In [ ]:
y_probs = clf.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_probs, average="macro", multi_class="ovo")
print('ROC Score is: ', roc_auc)

In [ ]:
# Defining Model
mnb = MultinomialNB()
# Training Model
mnb.fit(X_train, y_train)
# Making Predictions
y_pred6 = mnb.predict(X_test)
# Evaluating
print("Accuracy Score: ", accuracy_score(y_test, y_pred6))

In [ ]:
y_probs = mnb.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_probs, average="macro", multi_class="ovo")
print('ROC Score is: ', roc_auc)